## Importing FixDictionary

In [1]:
from FIXPyOrchestra import FixDictionary


## OrchestraXML class

This class is used to initiate fix variable - it has properties for fields, components, groups,..

1. Initiate fix instance
2. Take lines of messages - as a list of lists?
3. For each line (now a list), check for 35 field. If 35, flag true, start walking the dict

4. Get context from msgtype value. 
5. From context, get allowed values.
6. Set level = 1 - how? Save level as 3rd element in list? Yes for now
7. For next line, look in allowed values. 
8. If in allowed values - same level unless group (level+=1)
9. If not in allowed values, level-=1. Get last context? How!


In [36]:
# New new version - consolidating get_sub, defining self.props

class OrchestraXML():
    def __init__(self):
        '''Initialize fix class with fcgm properties'''
        # Initializes level, context_list, allowed, output message, context

        field = FixDictionary('fields')
        self.fields = (field.generateDictionary()) # [names, temp]

        component = FixDictionary('components')
        self.components = (component.generateDictionary()) # [names, miniIDs, miniGroup, miniComp, temp]

        group = FixDictionary('groups')
        self.groups = (group.generateDictionary()) # [names, numInGroup, miniIDs, miniGroup, miniComp, temp]

        message = FixDictionary('messages')
        self.messages = (message.generateDictionary()) # [names, miniIDs, miniGroup, miniComp, temp]

        self.level = 1
        self.context_list = []  # List of IDs that define context.
        self.context = []  # List of 2 items - tag, type(ie mcgf)
        self.allowed = []  # List of lists of allowed field,group,comp,numingrp
        self.output_string = []  # tag value level

    def get_allowed(self): 
        
        '''Takes current context value and type, and returns allowed fields, groups and components'''
        
        context = self.context[0]
        feature = self.context[1]
        if feature == 'm':
            allowed_fields = self.messages[context][1]
            allowed_groups = self.messages[context][2]
            allowed_comps = self.messages[context][3]
            allowed_numingroup = []
        elif feature == 'c':
            allowed_fields = self.components[context][1]
            allowed_groups = self.components[context][2]
            allowed_comps = self.components[context][3]
            allowed_numingroup = []
        else:
            allowed_numingroup = self.groups[context][1]
            allowed_fields = self.groups[context][2]
            allowed_groups = self.groups[context][3]
            allowed_comps = self.groups[context][4]
        return  ([allowed_fields, allowed_groups,allowed_comps, allowed_numingroup])

    def check_in_subfields(self, tv):
        '''Checks for tag in currently allowed fields'''
        return (tv[0] in self.allowed[0], '0')

    def check_in_subcomponents(self, tv):
        '''Checks for tag inside currently allowed components'''
        for c in self.allowed[2]:
            if tv[0] in self.components[c][1]:  # If tag is in component's fields
                return (True, c) # Return true and the component that contains this tag
        return (False, '0')

    def check_in_subgroups(self, tv):
        '''Checks for tag as numingroup currently allowed groups'''
        for g in self.allowed[1]:
            if tv[0] in self.groups[g][1]:  # If tag is in groups's numingroup
                return (True, g) # Return true and the group that contains this tag
        return (False, '0')

    def check_in_subs(self, tv):
        '''checks for tv in fields, subcomponents, subgroups. Appends output_str and returns True if found'''

        in_sub_field = self.check_in_subfields(tv)
        in_sub_comp = self.check_in_subcomponents(tv)
        in_sub_group = self.check_in_subgroups(tv)

        # Check in sub fields
        if in_sub_field[0]:
            self.out(tv, self.level)
            return True

        # Check in sub components
        elif in_sub_comp[0]:
            self.level += 1
            self.context = [in_sub_comp[1], 'c']
            self.context_list.append(self.context)
            self.allowed = self.get_allowed()
            self.out(tv, self.level)
            return True

        # Check in sub group
        elif in_sub_group[0]:
            self.level += 1
            self.context = [in_sub_group[1], 'g']
            self.context_list.append(self.context)
            self.allowed = self.get_allowed()
            self.out(tv, self.level-1)
            return True

        else:
            return False
        
    def out(self, tv, level):
        '''Outputs a list of tag, value and level to a list'''
        tv.append(level)
        self.output_string.append(tv)

    def walk(self, msg):
        '''Takes in 1 line of FIX message as list of lists, returns list of lists with levels appended'''

        flag35 = False

        for tv in msg:

            # For first 2 tags, level = 1. Level is appended to tag value list, index 2
            if not flag35:
                self.out(tv, self.level)

            # msgtype - sets context based on msgname, gets allowed fgc
            if tv[0] == '35':
                flag35 = True
                self.context = [tv[1], 'm']  # This is message type
                self.context_list.append(self.context)
                self.allowed = self.get_allowed()

            if flag35 and tv[0] != '35':

                bol = self.check_in_subs(tv)
                
                ct = 1
                while not bol and ct < 10:
                    ct += 1
                    self.level -= 1
                    self.context_list.pop()
                    self.context = self.context_list[-1]
                    self.allowed = self.get_allowed()
                    bol = self.check_in_subs(tv)

        return(self.output_string)


fix = OrchestraXML()
# fix.check_in_current_level('1024','2085')
# fix.check_context('2085')

# I got this from parse in prettyprinter
tagvalue_line_eg = pp.parse('fix_example_jim.txt')
firstline = tagvalue_line_eg[0]
fix.walk(firstline)

[['8', 'FIX.4.4', 1],
 ['9', '120', 1],
 ['34', '1', 1],
 ['35', '6', 1],
 ['23', 'myioiid123', 1],
 ['28', 'N', 1],
 ['55', 'TSLA', 2],
 ['453', '2', 1],
 ['448', 'LEIcode', 2],
 ['447', 'N', 2],
 ['452', '1', 2],
 ['448', 'biccode', 2],
 ['447', 'B', 2],
 ['452', '7', 2],
 ['802', '1', 2],
 ['523', 'ABC', 3],
 ['803', '1', 3],
 ['54', '1', 1],
 ['38', '100', 2],
 ['27', '100', 1],
 ['60', '2020062112:00:000.000', 1],
 ['44', '95.00', 1],
 ['10', '123\n', 2]]

8=FIX.4.4 <br />
9=120 <br />
34=1 <br />
35=6 <br />
23=myioiid123 <br />
28=N <br />
55=TSLA <br />
453=2 <br />
448=LEIcode <br />
447=N <br />
452=1 <br />
448=biccode <br />
447=B <br />
452=7 <br />
802=1 <br />
523=ABC <br />
803=1 <br />
54=1 <br />
38=100 <br />
27=100 <br />
60=2020062112:00:000.000 <br />
44=95.00 <br />
10=123 <br />



## Old iterations of OrchestraXML

In [171]:
# New new version - consolidating get_sub, defining self.props 

class OrchestraXML():
    def __init__(self):
        
        field = FixDictionary('fields')
        self.fields = (field.generateDictionary( ))
        #  [names, temp]
        
        component = FixDictionary('components')
        self.components = (component.generateDictionary( ))
        #  [names, miniIDs, miniGroup, miniComponent, temp]
        
        group = FixDictionary('groups')
        self.groups = (group.generateDictionary( ))
        #  [names, numInGroup, miniIDs, miniGroup, miniComponent, temp]
        
        message = FixDictionary('messages')
        self.messages = (message.generateDictionary( ))
        #  [names, miniIDs, miniGroup, miniComponent, temp]
        
        self.level = 1
        self.context_list = [] # List of IDs that define context.
        self.allowed = {}
        self.output_string = []
        self.current_context = [-1, 'm']
        
        
    def get_allowed(self): # feature is m c g
        # context = self.context_list[-1]
        context = self.current_context[0]
        feature = self.current_context[1]
        if feature == 'm':
            allowed_fields = self.messages[context][1]
            allowed_groups = self.messages[context][2]
            allowed_comps = self.messages[context][3]
            allowed_numingroup = []
            # return([allowed_fields,allowed_groups,allowed_comps])
        elif feature == 'c':
            allowed_fields = self.components[context][1]
            allowed_groups = self.components[context][2]
            allowed_comps = self.components[context][3]
            allowed_numingroup = []
            # return([allowed_fields,allowed_groups,allowed_comps])
        else:
            allowed_numingroup = self.groups[context][1]
            allowed_fields = self.groups[context][2]
            allowed_groups = self.groups[context][3]
            allowed_comps = self.groups[context][4]
        return([allowed_fields,allowed_groups,allowed_comps, allowed_numingroup])
    
    # Check each sub
    def check_in_subfields(self,tv):
        return (tv[0] in self.allowed[self.current_context[0]][0], '0')
    
    def check_in_subcomponents(self,tv):
        for c in self.allowed[self.current_context[0]][2]:
            if tv[0] in self.components[c][1]:
                return (True, c)
        return (False, '0')
    
    def check_in_subgroups(self,tv):
        for g in self.allowed[self.current_context[0]][1]:
            if tv[0] in self.groups[g][1]:
                return (True, g)
        return (False, '0')
    
    # Function that checks fcg for subs
    def check_in_subs(self,tv):
        
        # Checking subcomps for sub component elif loop
        in_sub_field = self.check_in_subfields(tv)
        in_sub_comp = self.check_in_subcomponents(tv)
        in_sub_group = self.check_in_subgroups(tv)
                
        # Testing zone - delete
                
        # Check in sub fields
        if in_sub_field[0]: # self.check_in_subfields(tv,allowed,current_context):
            tv.append(self.level)
            self.output_string.append(tv)
            return True
            
        # Check in sub components
        elif in_sub_comp[0]:
            self.level += 1
            self.current_context[0] = in_sub_comp[1]
            self.current_context[1] = 'c'
            self.context_list.append(self.current_context)
            self.allowed[self.current_context[0]] = self.get_allowed()
            tv.append(self.level)
            self.output_string.append(tv)
            return True
        
        # Check in sub group
        elif in_sub_group[0]:
            self.level += 1
            current_context = in_sub_group[1]
            self.context_list.append(current_context)
            self.allowed[self.current_context[0]] = self.get_allowed()
            # print(current_context)
            tv.append(self.level)
            self.output_string.append(tv)
            return True
        
        else:
            # self.output_string.append(tv)
            self.output_string.append('what do we do here')
            return False
    
    
    def walk(self, msg):
        '''Takes in 1 line of FIX message as list of lists, returns list of lists with levels appended'''
        
        # context_list = [] # List of IDs that define context.
        # allowed = {}
        flag35 = False
        # output_string = [] 
        # level = 1
        for tv in msg:
            print(self.context_list)
            # For first 2 tags, level = 1. Level is appended to tag value list, index 2
            if not flag35:
                tv.append(self.level)
                self.output_string.append(tv)
            
            # msgtype - sets context based on msgname, gets allowed fgc
            if tv[0]=='35':
                flag35 = True
                self.current_context = [tv[1],'m'] # This is message type
                self.context_list.append(self.current_context)
                self.allowed[self.current_context[0]] = self.get_allowed()
                
            if flag35 and tv[0]!='35':
                
                bol = self.check_in_subs(tv)
                while not bol:
                    self.context_list.pop()
                    print(self.context_list)
                    self.current_context = self.context_list[-1]
                    self.allowed[self.current_context[0]] = self.get_allowed()
                    bol = self.check_in_subs(tv)
                    
#                 # Checking subcomps for sub component elif loop
#                 in_sub_field = self.check_in_subfields(tv,allowed,current_context)
#                 in_sub_comp = self.check_in_subcomponents(tv,allowed,current_context)
#                 in_sub_group = self.check_in_subgroups(tv,allowed,current_context)
                
#                 # Testing zone - delete
                
#                 # Check in sub fields
#                 if in_sub_field[0]: # self.check_in_subfields(tv,allowed,current_context):
#                     tv.append(level)
#                     output_string.append(tv)
                
                
#                 # Check in sub components
#                 elif in_sub_comp[0]:
#                     level += 1
#                     current_context = in_sub_comp[1]
#                     context_list.append(current_context)
#                     allowed[current_context] = self.get_allowed(current_context,'c')
#                     tv.append(level)
#                     output_string.append(tv)
                    
#                 elif in_sub_group[0]:
#                     level += 1
#                     current_context = in_sub_group[1]
#                     allowed[current_context] = self.get_allowed(current_context,'g')
#                     # print(current_context)
#                     tv.append(level)
#                     output_string.append(tv)
                    
                
                
                # else:
#                     recursion_flag = True
#                     while not recursion_flag:
                        
                        # context_list.pop()
                        # current_context = context_list(-1)
                        # self.walk()
                    
                    ## output_string.append(f'{tv[0]} or here')
            
        
        return(self.output_string)
    
    
        
        
fix = OrchestraXML() 
# fix.check_in_current_level('1024','2085')
# fix.check_context('2085')

tagvalue_line_eg = pp.parse('fix_example_jim.txt')
firstline = tagvalue_line_eg[0]
fix.walk(firstline)




[]
[]
[]
[]
[['6', 'm']]
[['6', 'm']]
[['6', 'm']]
[['1003', 'c'], ['1003', 'c']]
[['1003', 'c']]
[]


IndexError: list index out of range

In [159]:
# New version - after ipad algo

class OrchestraXML():
    def __init__(self):
        
        field = FixDictionary('fields')
        self.fields = (field.generateDictionary( ))
        #  [names, temp]
        
        component = FixDictionary('components')
        self.components = (component.generateDictionary( ))
        #  [names, miniIDs, miniGroup, miniComponent, temp]
        
        group = FixDictionary('groups')
        self.groups = (group.generateDictionary( ))
        #  [names, numInGroup, miniIDs, miniGroup, miniComponent, temp]
        
        message = FixDictionary('messages')
        self.messages = (message.generateDictionary( ))
        #  [names, miniIDs, miniGroup, miniComponent, temp]
        
        self.level = 1
        self.context_list = [] # List of IDs that define context.
        self.allowed = {}
        self.output_string = []
        
        
    def get_allowed(self,context,feature): # m c g
        if feature == 'm':
            allowed_fields = self.messages[context][1]
            allowed_groups = self.messages[context][2]
            allowed_comps = self.messages[context][3]
            allowed_numingroup = []
            # return([allowed_fields,allowed_groups,allowed_comps])
        elif feature == 'c':
            allowed_fields = self.components[context][1]
            allowed_groups = self.components[context][2]
            allowed_comps = self.components[context][3]
            allowed_numingroup = []
            # return([allowed_fields,allowed_groups,allowed_comps])
        else:
            allowed_numingroup = self.groups[context][1]
            allowed_fields = self.groups[context][2]
            allowed_groups = self.groups[context][3]
            allowed_comps = self.groups[context][4]
        return([allowed_fields,allowed_groups,allowed_comps, allowed_numingroup])
    
    # Check each sub
    def check_in_subfields(self,tv,allowed,current_context):
        return (tv[0] in allowed[current_context][0], '0')
    
    def check_in_subcomponents(self,tv,allowed,current_context):
        for c in allowed[current_context][2]:
            if tv[0] in self.components[c][1]:
                return (True, c)
        return (False, '0')
    
    def check_in_subgroups(self,tv,allowed,current_context):
        for g in allowed[current_context][1]:
            if tv[0] in self.groups[g][1]:
                return (True, g)
        return (False, '0')
    
    # Function that checks fcg for subs
    def check_in_subs(self,tv,allowed,current_context,output_string,context_list):
        # Checking subcomps for sub component elif loop
        in_sub_field = self.check_in_subfields(tv,allowed,current_context)
        in_sub_comp = self.check_in_subcomponents(tv,allowed,current_context)
        in_sub_group = self.check_in_subgroups(tv,allowed,current_context)
                
        # Testing zone - delete
                
        # Check in sub fields
        if in_sub_field[0]: # self.check_in_subfields(tv,allowed,current_context):
            tv.append(self.level)
            output_string.append(tv)
                
                
        # Check in sub components
        elif in_sub_comp[0]:
            self.level += 1
            current_context = in_sub_comp[1]
            context_list.append(current_context)
            allowed[current_context] = self.get_allowed(current_context,'c')
            tv.append(self.level)
            output_string.append(tv)
                    
        elif in_sub_group[0]:
            self.level += 1
            current_context = in_sub_group[1]
            allowed[current_context] = self.get_allowed(current_context,'g')
            # print(current_context)
            tv.append(self.level)
            output_string.append(tv)
        else:
            output_string.append(tv)
    
    
    def walk(self, msg):
        '''Takes in 1 line of FIX message as list of lists, returns list of lists with levels appended'''
        
        context_list = [] # List of IDs that define context.
        allowed = {}
        flag35 = False
        output_string = [] 
        # level = 1
        for tv in msg:
            
            # For first 2 tags, level = 1. Level is appended to tag value list, index 2
            if not flag35:
                tv.append(self.level)
                output_string.append(tv)
            
            # msgtype - sets context based on msgname, gets allowed fgc
            if tv[0]=='35':
                flag35 = True
                current_context = tv[1] # This is message type
                context_list.append(current_context)
                allowed[current_context] = self.get_allowed(current_context,'m')
                
            if flag35 and tv[0]!='35':
                
                self.check_in_subs(tv,allowed,current_context,output_string,context_list)
                
#                 # Checking subcomps for sub component elif loop
#                 in_sub_field = self.check_in_subfields(tv,allowed,current_context)
#                 in_sub_comp = self.check_in_subcomponents(tv,allowed,current_context)
#                 in_sub_group = self.check_in_subgroups(tv,allowed,current_context)
                
#                 # Testing zone - delete
                
#                 # Check in sub fields
#                 if in_sub_field[0]: # self.check_in_subfields(tv,allowed,current_context):
#                     tv.append(level)
#                     output_string.append(tv)
                
                
#                 # Check in sub components
#                 elif in_sub_comp[0]:
#                     level += 1
#                     current_context = in_sub_comp[1]
#                     context_list.append(current_context)
#                     allowed[current_context] = self.get_allowed(current_context,'c')
#                     tv.append(level)
#                     output_string.append(tv)
                    
#                 elif in_sub_group[0]:
#                     level += 1
#                     current_context = in_sub_group[1]
#                     allowed[current_context] = self.get_allowed(current_context,'g')
#                     # print(current_context)
#                     tv.append(level)
#                     output_string.append(tv)
                    
                
                
                # else:
#                     recursion_flag = True
#                     while not recursion_flag:
                        
#                         context_list.pop()
#                         current_context = context_list(-1)
#                         self.walk()
                    
                    ## output_string.append(f'{tv[0]} or here')
            
        
        return(output_string)
    
    
        
        
fix = OrchestraXML() 
# fix.check_in_current_level('1024','2085')
# fix.check_context('2085')

tagvalue_line_eg = pp.parse('fix_example_jim.txt')
firstline = tagvalue_line_eg[0]
fix.walk(firstline)





[['8', 'FIX.4.4', 1],
 ['9', '120', 1],
 ['34', '1', 1],
 ['35', '6', 1],
 ['23', 'myioiid123', 1],
 ['28', 'N', 1],
 ['55', 'TSLA', 2],
 ['453', '2', 3],
 ['448', 'LEIcode'],
 ['447', 'N'],
 ['452', '1'],
 ['448', 'biccode'],
 ['447', 'B'],
 ['452', '7'],
 ['802', '1'],
 ['523', 'ABC'],
 ['803', '1'],
 ['54', '1', 3],
 ['38', '100', 4],
 ['27', '100', 4],
 ['60', '2020062112:00:000.000', 4],
 ['44', '95.00', 4],
 ['10', '123\n', 5]]

In [34]:
# Old version - Wed Jul 8 11:01pm

class OrchestraXML():
    def __init__(self):
        
        # codeset = FixDictionary('codeSets')
        # self.codesets = (codeset.generateDictionary( ))
        # datatype = FixDictionary('datatypes')
        # self.datatypes = (datatype.generateDictionary( ))
        
        field = FixDictionary('fields')
        self.fields = (field.generateDictionary( ))
        #  [names, temp]
        
        component = FixDictionary('components')
        self.components = (component.generateDictionary( ))
        #  [names, miniIDs, miniGroup, miniComponent, temp]
        
        group = FixDictionary('groups')
        self.groups = (group.generateDictionary( ))
        #  [names, numInGroup, miniIDs, miniGroup, miniComponent, temp]
        
        message = FixDictionary('messages')
        self.messages = (message.generateDictionary( ))
        #  [names, miniIDs, miniGroup, miniComponent, temp]
        
    # def get_allowed_contents_for_message(self,msgname):
    #     allowed_fields = self.messages[context][1]
    #     allowed_groups = self.messages[context][2]
    #     allowed_comps = self.messages[context][3]
    #     return allowed_fields,allowed_groups,allowed_comps
    
    def walk(self, msg):
        flag35 = False
        output_string = [] # Will hold 
        for tv in msg:
            if tv[0]=='35':
                flag35 = True
                context = tv[1] # This is message type
                allowed_fields = self.messages[context][1]
                allowed_groups = self.messages[context][2]
                allowed_comps = self.messages[context][3]
                # allowed_fields,allowed_groups,allowed_comps = self.get_allowed_contents_for_message[msgname]
                # print(allowed_fields,allowed_groups,allowed_comps)
            if flag35:
                print(tv)
    
    
    def check_context(self, tag):
        for component in self.components:
            # print(self.components[component])
            # Component has name,field,group,component,xx
            subfields = self.components[component][1]
            subgroups = self.components[component][2]
            subcomps = self.components[component][3]
            if  tag in subfields:
                print('here, in subcomponent ',component)
                context = component
                contexttype = 'component'
                break
            elif tag in subgroups:
                print('here, in subgroup',component)
                context = component
                contexttype = 'subgroup'
                break
        # if tag in self.components:
        #     print(f'it is in {self.components}')
        
        
    def check_in_current_level(self,currentname,checkname): # RIDICULOUS METHOD NAME
        # Notes: Check in components/groups. Fields have just name,desc,..
        startflag = False
        if not startflag:
            context = '1024' # StandardHeader component
            startflag = True # Basically, a new message
            level = 1
        
        if checkname in self.components[context][1]: # fields
            print('here')
            # Level remains the same
        
        elif checkname in self.groups[context][2]:
            print('in a subgroup')
            level += 1
        else:
            pass
        
    # def check_in_sublevels()
        
        
fix = OrchestraXML() 
# fix.check_in_current_level('1024','2085')
# fix.check_context('2085')


# Pretty Printer

## Consolidated PrettyPrinter

init - initializes fix class, assigns dictionaries. 3sec

prettify - final output

    parse - gets tag value pairs for all messages in a log file
        get_delim - gets delimiter
        
    prettyprint - For each line, gets levels, prints tag, value with names
        get_level - gets levels for all tags
            get_allowed - gets allowed list of fields, components, groups
            check_in_subs - checks tag in each sub
                check_in_subfields - checks sub fields, returns boolean, '0'
                check_subcomponents - checks sub components, returns boolean, subcomponent id
                check_in_subgroups - checks sub groups, returns boolean, subgroup id
                out - takes tag, value, level, returns list

In [95]:
# This needs to be optimized

class PrettyPrinter():
        
    def __init__(self):
        '''Initialize fix with fcgm properties'''
        
        # Initializes level, context_list, allowed, output message, context

        field = FixDictionary('fields')
        self.fields = (field.generateDictionary()) # [names, temp]
        
        component = FixDictionary('components')
        self.components = (component.generateDictionary()) # [names, miniIDs, miniGroup, miniComp, temp]
        
        group = FixDictionary('groups')
        self.groups = (group.generateDictionary()) # [names, numInGroup, miniIDs, miniGroup, miniComp, temp]
       
        message = FixDictionary('messages')
        self.messages = (message.generateDictionary()) # [names, miniIDs, miniGroup, miniComp, temp]
        
        self.level = 1
        self.context_list = []  # List of IDs that define context.
        self.context = []  # List of 2 items - tag, type(ie mcgf)
        self.allowed = []  # List of lists of allowed field,group,comp,numingrp
        self.output_string = []  # tag value level
        
    def get_allowed(self): 
        '''Takes current context value and type, and returns allowed fields, groups and components'''
        
        context = self.context[0]
        feature = self.context[1]
        if feature == 'm':
            allowed_fields = self.messages[context][1]
            allowed_groups = self.messages[context][2]
            allowed_comps = self.messages[context][3]
            allowed_numingroup = []
        elif feature == 'c':
            allowed_fields = self.components[context][1]
            allowed_groups = self.components[context][2]
            allowed_comps = self.components[context][3]
            allowed_numingroup = []
        else:
            allowed_numingroup = self.groups[context][1]
            allowed_fields = self.groups[context][2]
            allowed_groups = self.groups[context][3]
            allowed_comps = self.groups[context][4]
        return  ([allowed_fields, allowed_groups,allowed_comps, allowed_numingroup])

    def check_in_subfields(self, tv):
        '''Checks if the tag is in currently allowed fields'''
        return (tv[0] in self.allowed[0], '0')

    def check_in_subcomponents(self, tv):
        '''Checks if the tag is inside currently allowed components'''
        for c in self.allowed[2]:
            if tv[0] in self.components[c][1]:  # If tag is in component's fields
                return (True, c) # Return true and the component that contains this tag
        return (False, '0')

    def check_in_subgroups(self, tv):
        '''Checks if the tag is a numingroup inside currently allowed groups'''
        for g in self.allowed[1]:
            if tv[0] in self.groups[g][1]:  # If tag is in groups's numingroup
                return (True, g) # Return true and the group that contains this tag
        return (False, '0')

    def check_in_subs(self, tv):
        '''checks for tv in fields, subcomponents, subgroups. Appends output_str and returns True if found'''

        in_sub_field = self.check_in_subfields(tv)
        in_sub_comp = self.check_in_subcomponents(tv)
        in_sub_group = self.check_in_subgroups(tv)

        # Check in sub fields
        if in_sub_field[0]:
            self.out(tv, self.level,'field')
            return True

        # Check in sub components
        elif in_sub_comp[0]:
            self.level += 1
            self.context = [in_sub_comp[1], 'c']
            self.context_list.append(self.context)
            self.allowed = self.get_allowed()
            self.out(tv, self.level,'component',self.components[self.context[0]][0])
            return True

        # Check in sub group
        elif in_sub_group[0]:
            self.level += 1
            self.context = [in_sub_group[1], 'g']
            self.context_list.append(self.context)
            self.allowed = self.get_allowed()
            self.out(tv, self.level-1,'numingrp',self.groups[self.context[0]][0])
            return True

        else:
            return False
        
    def out(self, tv, level, typee, name = None):
        '''Appends tag, value, level, type and name to tag value list'''
        tv.append(level)
        tv.append(typee)
        if name:
            tv.append(name)
        self.output_string.append(tv)

    def get_level(self, msg):
        '''Takes in 1 line of FIX message as list of lists, returns list of lists with levels appended'''

        flag35 = False

        for tv in msg:

            # For StandardHeader - level = 1. Level is appended to tag value list, index 2
            if not flag35:
                self.out(tv, self.level, 'component')

            # msgtype - sets context based on msgname, gets allowed fgc
            if tv[0] == '35':
                flag35 = True
                self.context = [tv[1], 'm']  # type - message
                self.context_list.append(self.context)
                self.allowed = self.get_allowed()

            if flag35 and tv[0] != '35':

                bol = self.check_in_subs(tv)
                
                # Go back one context if tag not found in current context
                # Setting a count to avoid infinite loops
                ct = 1
                while not bol and ct < 30:
                    ct += 1
                    self.level -= 1
                    if self.level < 1:
                        print(f'Out of bounds! Check tag {tv[0]}')
                        break
                    self.context_list.pop()
                    self.context = self.context_list[-1]
                    self.allowed = self.get_allowed()
                    bol = self.check_in_subs(tv)
        return(self.output_string)
    
    def prettify(self,logfile):
        '''COnsolidated function. Takes logfine as input, pretty prints messages'''
        outp = self.parse(logfile)
        self.prettyprint(outp)
    
    def parse(self, logfile):
        '''Parses log file, returns list of tag values in list of tag value lines'''
        with open(logfile, 'r') as f:
            lines = f.readlines()
            delim = self.get_delim(lines)
            tagvalue_lines = []
            for line in lines:
                
                line = line.replace('\u2028447','') # Replaces this pesky unicode
                strt_idx = line.find('8=FIX')
                line=line[strt_idx:]
                columns = line.split(delim)
                tagvalue = []
                for col in columns:
                    values = col.split('=')
                    if len(values) > 1: # This ignores all \n
                        tagvalue.append([values[0],values[1]])
                tagvalue_lines.append(tagvalue)
        return tagvalue_lines
    
    def get_delim(self, lines):  
        '''Finds delimiter for given FIX messages'''
        i = 0
        delim = ''
        while delim not in ['<SOH>',':',';','^',' ','|'] and i<10:
            try:
                if '<SOH>' in lines[i]:
                    delim = '<SOH>'
                else:
                    delim_idx = lines[i].rfind('10=')
                    delim = lines[i][delim_idx-1]
            except:
                print("Error: couldn't find delimiter in line ", i)
            i+=1            
        return delim
    
    def prettyprint(self,tagvalue_lines):
        '''Takes in levels, types and name for tags and values, prints output'''
        
        for line in tagvalue_lines:
            print() # Blank line
            print('New message')
            print('\tStandardHeader component')
            
            line_with_level = self.get_level(line)
            
            for tagvalue in line_with_level:
                try:
                   
                    key = tagvalue[0]
                    level = tagvalue[2]
                    keyname = self.fields[key][0]
                    
                    if key == '35': # If message
                        try:
                            msg_name = self.messages[tagvalue[1]][0]
                            print(level*'\t'+'  '+f'{keyname}({key}) = {msg_name}({tagvalue[1]})')
                        except KeyError:
                            print(f'No message name listed for 35 = {tagvalue[1]}')
                            print(level*'\t'+f'{keyname}({key}) = ({tagvalue[1]})')
                    else:
                        if tagvalue[3] == 'component':
                            if level <= 1: # Applicable to level 1 components only
                                level = 2
                            if len(tagvalue)>4:
                                print((level-1)*'\t'+f'{tagvalue[4]} component') # Component name
                            print((level-1)*'\t'+'  '+f'{keyname}({key}) = {tagvalue[1]}')
                        elif tagvalue[3] == 'numingrp':
                            print(level*'\t'+f'{tagvalue[4]} group') # Group name
                            print(level*'\t'+f'{keyname}({key}) = {tagvalue[1]}')
                        else:
                            print(level*'\t'+f'{keyname}({key}) = {tagvalue[1]}')
                    
                except:
                    print(f'key name not found for tag {key}')
            self.level = 1 # reset level after every message

In [96]:
pp = PrettyPrinter()

In [97]:
tagvalue_line_eg = pp.prettify('fix_example_jim.txt')


New message
	StandardHeader component
	  BeginString(8) = FIX.4.4
	  BodyLength(9) = 120
	  MsgSeqNum(34) = 1
	  MsgType(35) = IOI(6)
	IOIID(23) = myioiid123
	IOITransType(28) = N
	Instrument component
	  Symbol(55) = TSLA
	Parties group
	NoPartyIDs(453) = 2
		PartyID(448) = LEIcode
		PartyIDSource(447) = N
		PartyRole(452) = 1
		PartyID(448) = biccode
		PartyIDSource(447) = B
		PartyRole(452) = 7
		PtysSubGrp group
		NoPartySubIDs(802) = 1
			PartySubID(523) = ABC
			PartySubIDType(803) = 1
	Side(54) = 1
	OrderQtyData component
	  OrderQty(38) = 100
	IOIQty(27) = 100
	TransactTime(60) = 2020062112:00:000.000
	Price(44) = 95.00
	StandardTrailer component
	  CheckSum(10) = 123


New message
	StandardHeader component
	  BeginString(8) = FIX.4.4
	  BodyLength(9) = 120
	  MsgSeqNum(34) = 1
	  MsgType(35) = IOI(6)
	IOIID(23) = myioiid123
	IOITransType(28) = N
	Instrument component
	  Symbol(55) = TSLA
	Parties group
	NoPartyIDs(453) = 2
		PartyID(448) = LEIcode
		PartyIDSource(447) = N
		P

In [24]:
# tagvalue_lines = pp.prettify('logtest_soh_single.txt')

### Backup - working

In [ ]:
# Backup working 

# This needs to be optimized
import time # For optimization, to be deleted

class PrettyPrinter():
        
    def __init__(self):
        '''Initialize fix with fcgm properties'''
        
        # Initializes level, context_list, allowed, output message, context

        field = FixDictionary('fields')
        self.fields = (field.generateDictionary()) # [names, temp]
        
        component = FixDictionary('components')
        self.components = (component.generateDictionary()) # [names, miniIDs, miniGroup, miniComp, temp]
        
        group = FixDictionary('groups')
        self.groups = (group.generateDictionary()) # [names, numInGroup, miniIDs, miniGroup, miniComp, temp]
       
        message = FixDictionary('messages')
        self.messages = (message.generateDictionary()) # [names, miniIDs, miniGroup, miniComp, temp]
        
        self.level = 1
        self.context_list = []  # List of IDs that define context.
        self.context = []  # List of 2 items - tag, type(ie mcgf)
        self.allowed = []  # List of lists of allowed field,group,comp,numingrp
        self.output_string = []  # tag value level
        
    def get_allowed(self): 
        
        '''Takes current context value and type, and returns allowed fields, groups and components'''
        
        context = self.context[0]
        feature = self.context[1]
        if feature == 'm':
            allowed_fields = self.messages[context][1]
            allowed_groups = self.messages[context][2]
            allowed_comps = self.messages[context][3]
            allowed_numingroup = []
        elif feature == 'c':
            allowed_fields = self.components[context][1]
            allowed_groups = self.components[context][2]
            allowed_comps = self.components[context][3]
            allowed_numingroup = []
        else:
            allowed_numingroup = self.groups[context][1]
            allowed_fields = self.groups[context][2]
            allowed_groups = self.groups[context][3]
            allowed_comps = self.groups[context][4]
        return  ([allowed_fields, allowed_groups,allowed_comps, allowed_numingroup])

    def check_in_subfields(self, tv):
        '''Checks for tag in currently allowed fields'''
        return (tv[0] in self.allowed[0], '0')

    def check_in_subcomponents(self, tv):
        '''Checks for tag inside currently allowed components'''
        for c in self.allowed[2]:
            if tv[0] in self.components[c][1]:  # If tag is in component's fields
                return (True, c) # Return true and the component that contains this tag
        return (False, '0')

    def check_in_subgroups(self, tv):
        '''Checks for tag as numingroup currently allowed groups'''
        for g in self.allowed[1]:
            if tv[0] in self.groups[g][1]:  # If tag is in groups's numingroup
                return (True, g) # Return true and the group that contains this tag
        return (False, '0')

    def check_in_subs(self, tv):
        '''checks for tv in fields, subcomponents, subgroups. Appends output_str and returns True if found'''

        in_sub_field = self.check_in_subfields(tv)
        in_sub_comp = self.check_in_subcomponents(tv)
        in_sub_group = self.check_in_subgroups(tv)

        # Check in sub fields
        if in_sub_field[0]:
            self.out(tv, self.level)
            return True

        # Check in sub components
        elif in_sub_comp[0]:
            self.level += 1
            self.context = [in_sub_comp[1], 'c']
            self.context_list.append(self.context)
            self.allowed = self.get_allowed()
            self.out(tv, self.level)
            return True

        # Check in sub group
        elif in_sub_group[0]:
            self.level += 1
            self.context = [in_sub_group[1], 'g']
            self.context_list.append(self.context)
            self.allowed = self.get_allowed()
            self.out(tv, self.level-1)
            return True

        else:
            return False
        
    def out(self, tv, level):
        '''Outputs a list of tag, value and level to a list'''
        tv.append(level)
        self.output_string.append(tv)

    def get_level(self, msg):
        '''Takes in 1 line of FIX message as list of lists, returns list of lists with levels appended'''

        flag35 = False

        for tv in msg:

            # For first 2 tags, level = 1. Level is appended to tag value list, index 2
            if not flag35:
                self.out(tv, self.level)

            # msgtype - sets context based on msgname, gets allowed fgc
            if tv[0] == '35':
                flag35 = True
                self.context = [tv[1], 'm']  # This is message type
                self.context_list.append(self.context)
                self.allowed = self.get_allowed()

            if flag35 and tv[0] != '35':

                bol = self.check_in_subs(tv)
                
                ct = 1
                while not bol and ct < 10:
                    ct += 1
                    self.level -= 1
                    self.context_list.pop()
                    self.context = self.context_list[-1]
                    self.allowed = self.get_allowed()
                    bol = self.check_in_subs(tv)
        # print(self.output_string)
        return(self.output_string)
    
    def prettify(self,logfile):
        outp = self.parse(logfile)
        self.prettyprint(outp)
    
    def parse(self, logfile):
        '''Parses log file, returns list of tag values in list of tag value lines'''
        with open(logfile, 'r') as f:
            lines = f.readlines()
            delim = self.get_delim(lines)
            tagvalue_lines = []
            for line in lines:
                
                line = line.replace('\u2028447','') # Replaces this pesky unicode
                strt_idx = line.find('8=FIX')
                line=line[strt_idx:]
                columns = line.split(delim)
                tagvalue = []
                for col in columns:
                    values = col.split('=')
                    if len(values) > 1: # This ignores all \n
                        tagvalue.append([values[0],values[1]])
                tagvalue_lines.append(tagvalue)
        return tagvalue_lines
    
    def get_delim(self, lines):  
        '''Finds delimiter for given FIX messages'''
        i = 0
        delim = ''
        while delim not in ['<SOH>',':',';','^',' ','|'] and i<10:
            try:
                if '<SOH>' in lines[i]:
                    delim = '<SOH>'
                else:
                    delim_idx = lines[i].rfind('10=')
                    delim = lines[i][delim_idx-1]
            except:
                print("Error: couldn't find delimiter in line ", i)
            i+=1            
        return delim
    
    def prettyprint(self,tagvalue_lines):
        
        
        for line in tagvalue_lines:
            print() # Blank line
            print('New message')
            
            line_with_level = self.get_level(line)
            
            for tagvalue in line_with_level:
                try:
                    key = tagvalue[0]
                    level = tagvalue[2]
                    keyname = self.fields[key][0]
                    # print(key, keyname) # NEED TO SAVE BOTH KEY AND KEYNAME
                    
                    if key == '35':
                        try:
                            msg_name = self.messages[tagvalue[1]][0]
                            print(level*'\t'+f'{keyname}({key}) = {msg_name}({tagvalue[1]})')
                        except KeyError:
                            print(f'No message name listed for 35 = {tagvalue[1]}')
                            print(level*'\t'+f'{keyname}({key}) = ({tagvalue[1]})')
                    else:
                        print(level*'\t'+f'{keyname}({key}) = {tagvalue[1]}')
                    
                except:
                    print(f'key name not found for tag {key}')
            self.level = 1

## New pretty printer class

In [14]:
# Possible deletion

class ParsedLine():
    def __init__(self, line):
        self.tag = line[0]
        self.value = line[2]
        self.tagname = line[1]
        self.tab = 1
        if len(line)==4:
            self.valuename = line[3]

line = ['52','10','asd','sdf']
a = (ParsedLine(line))
print(a.value)

asd


In [4]:
class PrettyPrinter():
    
    
    def parse(self, logfile):
        with open(logfile, 'r') as f:
            lines = f.readlines()
            delim = self.get_delim(lines)
            tagvalue_lines = []
            for line in lines:
                
                line = line.replace('\u2028447','')
                strt_idx = line.find('8=FIX')
                line=line[strt_idx:]
                columns = line.split(delim)
                tagvalue = []
                for col in columns:
                    values = col.split('=')
                    if len(values) > 1: # This ignores all \n
                        tagvalue.append([values[0],values[1]])
                tagvalue_lines.append(tagvalue)
        return tagvalue_lines
    
        
    
    def prettyprint(self,tagvalue_lines,fix):
        for line in tagvalue_lines:
            for tagvalue in line:
                try:
                    key = tagvalue[0]
                    keyname = fix.fields[key][0]
                    # print(key, keyname) # NEED TO SAVE BOTH KEY AND KEYNAME
                    
                    if key == '35':
                        try:
                            msg_name = fix.messages[tagvalue[1]][0]
                            print(f'{keyname}({key}) = {msg_name}({tagvalue[1]})')
                        except KeyError:
                            print(f'No message name listed for 35 = {tagvalue[1]}')
                            print(f'{keyname}({key}) = ({tagvalue[1]})')
                    else:
                        print(f'{keyname}({key}) = {tagvalue[1]}')


                    
                    
                except:
                    print(f'key name not found for tag {key}')
        
        
    def work(self, logfile):
        tv = self.parse(logfile)
        self.prettyprint(tv, fix)
    
    def get_delim(self, lines):        
        i = 0
        delim = ''
        while delim not in ['<SOH>',':',';','^',' ','|'] and i<10:
            try:
                if '<SOH>' in lines[i]:
                    delim = '<SOH>'
                else:
                    delim_idx = lines[i].rfind('10=')
                    delim = lines[i][delim_idx-1]
            except:
                print("Error: couldn't find delimiter in line ", i)
            i+=1            
        return delim
    
    
# Test cases
pp = PrettyPrinter()

tagvalue_line_eg = pp.parse('fix_example_jim.txt')

# tagvalue_lines = pp.parse('logtest_colon_multi.txt')
# pp.prettyprint(tagvalue_lines,fix)

# tagvalue_lines = pp.parse('logtest_soh_single.txt')
# pp.prettyprint(tagvalue_lines,fix)

pp.work('logtest_soh_single_1.txt')
# pp.work('logtest_colon_few.txt')

pp.work('fix_example_jim.txt')

# tagvalue_lines = pp.parse('logtest_colon_few.txt')
# pp.prettyprint(tagvalue_lines,fix)
    
    

BeginString(8) = FIX.4.4
BodyLength(9) = 326
MsgType(35) = NewOrderSingle(D)
NoPartyIDs(453) = 3
PartyID(448) = DEU
PartyIDSource(447) = B
PartyRole(452) = 1
NoPartySubIDs(802) = 1
PartySubID(523) = A1
PartySubIDType(803) = 10
PartyID(448) = 104317
PartyIDSource(447) = H
PartyRole(452) = 83
PartyID(448) = GSI
PartyIDSource(447) = B
PartyRole(452) = 4
PartyRoleQualifier(2376) = 23
NoPartySubIDs(802) = 1
PartySubID(523) = C3
PartySubIDType(803) = 10
CheckSum(10) = 105
BeginString(8) = FIX.4.4
BodyLength(9) = 120
MsgSeqNum(34) = 1
MsgType(35) = IOI(6)
IOIID(23) = myioiid123
IOITransType(28) = N
Symbol(55) = TSLA
NoPartyIDs(453) = 2
PartyID(448) = LEIcode
PartyIDSource(447) = N
PartyRole(452) = 1
PartyID(448) = biccode
PartyIDSource(447) = B
PartyRole(452) = 7
NoPartySubIDs(802) = 1
PartySubID(523) = ABC
PartySubIDType(803) = 1
Side(54) = 1
OrderQty(38) = 100
IOIQty(27) = 100
TransactTime(60) = 2020062112:00:000.000
Price(44) = 95.00
CheckSum(10) = 123



## Original prettify

In [ ]:
def prettify(src_file, dest_file, dict, msg_dict, field_enum_dict, parse_enum):
    with open(dest_file, 'w') as fo: # REMOVED , 'wb' from open bracket
        with open(src_file, 'r') as f: # REMOVED , 'rb' from open bracket
            print()
            print(src_file)
            
            # Save all lines in src_file
            lines = f.readlines()
            i=-1
            # Get delimiter
            try:
                delim_idx = lines[0].rfind('10=')
                delim = '<SOH>'
                #delim = lines[0][delim_idx-1]
                print(f'The delimiter is {delim} \n')
            except:
                print('Could not identify delimiter in first line of the log file. \nTrying second line. \n')
                delim_idx = lines[1].rfind('10=')
                delim = lines[1][delim_idx-1]
                print(f'The delimiter is {delim} \n')
            
            for line in lines:
                i+=1
                # Find the start index in a line
                strt_idx = line.find('8=FIX')
                line=line[strt_idx:]
                
                # Get all columns from line with delimiter
                columns = line.split(delim)
                for col in columns:
                    
                    #print(col)
                    values = col.split('=')
                    if len(values) > 1: # This ignores all \n
                        try:
                            key = int(values[0])
                            tab = checksubgroup(key)
                            
                            tag_name = dict[key] # Gets tag name from dict
                            
                            # Getting message names from MessageType
                            if key == 35:
                                try:
                                    msg_name = msg_dict[values[1]]
                                    fo.write(tab+"%s(%d) = %s(%s)\n" % (tag_name, key, msg_name, values[1]))
                                except KeyError:
                                    print("msg_name KeyError: %d, value: %s, line_num: %d: tag 35 do not have %s msg listed" % (key, values[1], i, values[1]))
                                    fo.write(tab+"%s(%d) = %s\n" % (tag_name, key, values[1]))
                            
                            # Getting names from fields
                            elif parse_enum and key in field_enum_dict:
                                enum_dict = field_enum_dict[key]
                                
                                
                                
                                if values[1] in enum_dict:
                                    fo.write(tab+"%s(%d) = %s(%s)\n" % (tag_name, key, enum_dict[values[1]], values[1]))
                                else:
                                    fo.write(tab+"%s(%d) = %s\n" % (tag_name, key, values[1]))
                                    print("enum KeyError: %d, value: %s, line_num: %d: tag %d does not have this enum value in dictionary: %s" % (key, values[1], i, key, values[1]))
                            else:
                                fo.write(tab+"%s(%d) = %s\n" % (tag_name, key, values[1]))
                        except ValueError:
                            print(column)
                            fo.write("%s\n" % col)
                        except KeyError:
                            tab = checksubgroup(values[0])
                            fo.write(tab+"%s = %s\n" % (values[0], values[1]))
                            print("tag_name KeyError: %d, value: %s, line_num: %d" % (key, values[1], i))
                fo.write("\n\n")
            
            
            
                
                

src_log = 'logtest.txt'
#src_log2 = 'delimitertest.log'
#src_log3 = 'FIX.4.2-KOD-BON.messages.current.log'
#src_log4 = 'FIXT.1.1-EXEC-BANZAI.messages.log'




output_log = 'log2.txt'
prettify(src_log, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true
#prettify(src_log2, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true
#prettify(src_log3, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true
#prettify(src_log4, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true

In [ ]:
aa = '8=FIX.4.4<SOH>9=326<SOH>35=D<SOH>453=3<SOH>448=DEU<SOH>447=B<SOH>452=1<SOH>802=1<SOH>523=A1<SOH>803=10<SOH>448=104317<SOH> 447=H<SOH>452=83<SOH>448=GSI<SOH>447=B<SOH>452=4<SOH>2376=23<SOH>802=1<SOH>523=C3<SOH> 803=10<SOH>10=105'

print(aa.split('<SOH>'))

In [ ]:
with open('logtest_soh_single.txt') as f:
    lines = f.readlines()
    lines=lines[0]
    # print(lines)
    aa = lines.split('<SOH>')
    print(aa)